In [1]:
import json
import getpass
from requests import Session
from pprint import pprint
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

url = "https://localhost"
user = input()
password = getpass.getpass()

# LOGIN

In [2]:
session = Session()

first = session.get(f"{url}", verify=False)
csrftoken = first.cookies["csrftoken"]

data = json.dumps(
    {"username": user, "password": password, "csrfmiddlewaretoken": csrftoken}
)

headers = {
    "X-CSRFToken": first.headers["Set-Cookie"].split("=")[1].split(";")[0],
    "Referer": url,
    "X-Requested-With": "XMLHttpRequest",
}

req = session.post(
    f"{url}/api/auth/", data=data, cookies=first.cookies, headers=headers, verify=False
)
if req.status_code != 200:
    print(req.text)
    exit(1)

# GET DUMP LIST

In [4]:
dumps = session.get(f"{url}/api/dumps/", verify=False).json()
print(f"{len(dumps)} dumps found")
if dumps: 
    pprint(dumps[0])
    dump_pk = dumps[0]['index']

4 dumps found
{'author': 1,
 'color': '#FF0000',
 'description': None,
 'folder': None,
 'index': 'f430c72c-8afc-11ef-86cc-0242ac130009',
 'name': 's3',
 'operating_system': 'Windows',
 'status': 3,
 'upload': '/media/media/f430c72c-8afc-11ef-86cc-0242ac130009/sample003.bin'}


# CREATE DUMP

In [ ]:

files = {'upload': ('sorpresa.zip', open('/AMF_MemorySamples/linux/sorpresa.zip','rb'))}
data = {
    'payload': '{"operating_system": "Linux", "name": "sorpresa", "folder": {"name": "linux-samples"}}'
}
res = session.post(f"{url}/api/dumps/", files=files, data=data, cookies=first.cookies, headers=headers, verify=False)
if res.status_code == 200:
    pprint(res.json())
    dump_pk = res.json()["index"]
else:
    print(res.status_code, res.text)

# CREATE DUMP FROM REMOTE DUMP

In [ ]:
""" TODO
# This code requires a file on the server in the folder specified in the LOCAL_UPLOAD_PATH
# settings folder

files = {'upload': None}
data = {
    'payload': '{"operating_system": "Linux", "name": "remote-test", "folder": {"name": "linux-samples"}, "local_folder": "/uploads/sorpresa.vmem"}'
}
res = session.post(f"{url}/api/dumps/", files=files, data=data, cookies=first.cookies, headers=headers, verify=False)
if res.status_code == 200:
    pprint(res.json())
    dump_pk = res.json()["index"]
else:
    print(res.status_code, res.text)

# GET PLUGIN LIST

In [7]:
res = session.get(f"{url}/api/plugins/", verify=False)
if res.status_code == 200:
    plugins = res.json()
    print(f"{len(plugins)} plugins found")
    pprint(plugins[0])
else:
    print(res.status_code, res.text)    
res = session.get(f"{url}/api/plugins/?operating_system=Other", verify=False)
if res.status_code == 200:
    plugins = res.json()
    print(f"{len(plugins)} plugins found")
    pprint(plugins[0])
else:
    print(res.status_code, res.text)    

137 plugins found
{'clamav_check': False,
 'disabled': False,
 'local': False,
 'local_date': None,
 'local_dump': False,
 'maxmind_check': False,
 'name': 'timeliner.Timeliner',
 'operating_system': 'Other',
 'regipy_check': False,
 'vt_check': False}
8 plugins found
{'clamav_check': False,
 'disabled': False,
 'local': False,
 'local_date': None,
 'local_dump': False,
 'maxmind_check': False,
 'name': 'timeliner.Timeliner',
 'operating_system': 'Other',
 'regipy_check': False,
 'vt_check': False}


# GET PLUGINS FOR A SPECIFIC DUMP

In [ ]:
res = session.get(f"{url}/api/dumps/{dump_pk}/plugins", verify=False)
if res.status_code == 200:
    if example := res.json():
        print(example[0])
        plugin_pk = example[0]
else:
    print(res.status_code, res.text)        

# GET PLUGIN STATUS ON SELECTED DUMP

In [ ]:
""" TODO
res = session.get(f"{url}/api/dumps/{dump_pk}/plugins/{plugin_pk}/", verify=False)
if res.status_code == 200:
    pprint(res.json())
    result_pk = [x['pk'] for x in res.json() if x['plugin'] == 'linux.pslist.PsList'][0]
    print(res.status_code)
"""

# RESUBMIT PLUGIN

In [ ]:
""" TODO
res = session.post(f"{url}/api/dumps/{dump_pk}/results/{result_pk}/resubmit/", data={'parameter': {'dump': True}})
if res.status_code == 200:
    pprint(res.json())
else:
    print(res.status_code)
"""

# CHECK PLUGIN STATUS

In [ ]:
""" TODO
status = 'Running'
while status != 'Success':
    res = session.get(f"{url}/api/dumps/{dump_pk}/results/{result_pk}/")
    if res.status_code == 200:
        status = res.json()['status']
        pprint(status)
    else:
        print(res.status_code)
        break
"""

# SHOW PLUGIN RESULT

In [ ]:
""" TODO
res = session.get(f"{url}/api/dumps/{dump_pk}/results/{result_pk}/result")
if res.status_code == 200:
    pprint(len(res.json()))
else:
    print(res.status_code)
"""

# WORK WITH RETURNED DATA

In [ ]:
""" TODO
import pandas as pd
import pygwalker as pyg
df = pd.DataFrame.from_records(res.json())
walker = pyg.walk(df)
"""

# EDIT DUMP

In [ ]:
data = {
    "name": "string",
    "folder": {
        "name": "string"
    },
    "color": "#7218EE",
    "status": "1",
    "comment": "stringghjny"
}
res = session.patch(f"{url}/api/dumps/{dump_pk}", json=data, cookies=first.cookies, headers=headers, verify=False)
if res.status_code == 200:
    pprint(res.json())
    dump_pk = res.json()["index"]
else:
    print(res.status_code, res.text)

# DELETE DUMP

In [ ]:
res = session.delete(f"{url}/api/dumps/{dump_pk}", cookies=first.cookies, headers=headers, verify=False)
print(res.status_code, res.text)   